In [57]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import matplotlib as mpl
import matplotlib.colors as mcolors  
import matplotlib.pyplot as plt      

from matplotlib import rc 

import scipy

In [3]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        
    Returns:
        p_c: Critical pressure in Pa.\n
        v_c: Critical volume in m^3/mol.\n
        T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/k_B/N_A
    
    return p_c, v_c, T_c

In [4]:
def theoretical_isotherms(T_range, v_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """
    

    


    isotherms = {}
    #v_R = np.logspace(min(v_range), max(v_range), num = 500, base = 10.0)
    isotherms['v'] = v_range

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms[T] = p_R

    return isotherms

In [5]:
def update_isotherm(change):
    
    """    
        This function makes the isotherm expressed by a
        float ipython widget visible in the 'fig' plot and
        hides the rest of the lines (the ones in the list 
        'saved' are always displayed).\n

    Args:
        change: variable returned by my_float_widget.observe(update
        _isotherm, 'value').\n

    """
    
    temp = np.arange(T_r.min, T_r.max+0.05, 0.05)
    t_range = np.around(temp, 2)
    
    old, = np.where(t_range == float(change.old))[0]
    i, = np.where(t_range == float(change.new))[0]
    
    if old not in saved:
        fig.marks[old].visible = False
        fig.marks[old].display_legend = False
        
    fig.marks[i].visible = True
    fig.marks[i].display_legend = True
    
    update_tracer(change)

In [6]:
def save_isotherm(T):
    
    """    
        This function adds the isotherm expressed by a
        float ipython widget ('T_r') visible to the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        a: variable returned by save_button.on_click(save_isotherm)
        .\n

    """
    i = get_isotherm_index(T)
    saved.append(i)    
    fig.marks[i].visible = True  #trivial when called by 'save isotherm button' but allows to 'show all checkbox' work

def show_isotherm(T):
    i = get_isotherm_index(T)
    fig.marks[i].visible = True

In [7]:
def get_isotherm_index(T):
    t_range = get_t_range(t_min_input, t_max_input) 
    i, = np.where(t_range == float(T))[0]
    return i

In [8]:
def save_button_clicked(a):
    save_isotherm(T_r.value)

In [9]:
def undo(change):
    
    """    
        This function pops the isotherm expressed by a
        float ipython widget ('T_r') visible from the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        change: variable returned by undo_button.on_click(undo)
        .\n

    """
        
    if len(saved) > 0:
        popped = saved.pop(-1)
        i, = np.where(t_range == float(T_r.value))[0]
        if i != popped:
            fig.marks[popped].visible = False

In [10]:
def update_critics(change):
    
    if b.value == 0.0:
        return None
    
    p_c, v_c, T_c = calculate_critic(a.value, b.value * 0.001)
    
    p_c_text.value = '%.2f' % (p_c / 1000000.0) #Show in MPa
    v_c_text.value = '%.2f' % (v_c * 1000000.0)
    T_c_text.value = '%.2f' % (T_c)

In [11]:
def get_t_range(t_min_input, t_max_input, num_input):
    
    t_min = t_min_input.value
    t_max = t_max_input.value
    num = num_input.value
    
    if t_min > t_max or t_min == t_max:
        return None
    
    return np.linspace(start=t_min, stop=t_max, num=num)

In [12]:
def update_t_range(change):
    
    t_range = get_t_range(t_min_input, t_max_input)
    
    T_r.min = min(t_range)
    T_r.max = max(t_range)

    data = isotherms(a.value, b.value * 0.001, t_range, v_limits)
    
    #isotherm_lines = []
    marks = []
    size  = len(data.keys())
    step = np.floor_divide([255], [size])[0]

    c1 = 255
    c2 = 0
    c3 = 0
    
    vis = str(T_r.value)
    
    if  vis not in data.keys():
        
        vis = '1.00'

        if vis not in data.keys():
            vis = min(data.keys())

    T_r.value = float(vis)
    
    for line in fig.marks:
        line.visible = False
    
    saved.clear()
    
    for t in data.keys():
        if t != 'v':

            color = '#%02x%02x%02x' % (c1, c2, c3)

            marks.append(bqm.Lines(
            #isotherm_lines.append(bqm.Lines(
                x = data.get('v'), 
                y = data.get(t), 
                scales = {'x': scale_x, 'y': scale_y}, 
                #opacities = [1.0],
                visible = t == vis,
                display_legend = t == vis,
                colors = [color],
                labels = ['T = ' + t],
                ))
            c2 = c2 + step
            

    marks.append(tracer)
    marks.append(labels_points)
    marks.append(isopressure)
       
    maxwell_construction_checkbox.value = False
    show_tracer_checkbox.value = False
    
    fig.marks = marks

In [13]:
def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [117]:
def show_tracer(change):
    
    if show_tracer_checkbox.value:
        tracer.visible = True
        v_slider.disabled = False
        update_tracer(change)
        
        label_input.disabled = False
        add_label_button.disabled = False
    else:
        tracer.visible = False
        v_slider.disabled = True
        
        label_input.disabled = True
        add_label_button.disabled = True

In [126]:
def update_tracer(change):
    
    v = v_slider.value
    T = T_slider.value
    
    for mark in fig.marks:
        
        if mark.labels[0] == str(T):

            x_values = mark.x
            y_values = mark.y
            break
    
    i_v = find_nearest_index(x_values, v)
    tracer.x = np.array([x_values[i_v]])
    tracer.y = np.array([y_values[i_v]])

In [16]:
def add_label(x, y, label='froga'):
    
    x_labels = [elem for elem in labels_points.x]
    y_labels = [elem for elem in labels_points.y]
    
    x_labels.append(tracer.x[0])
    y_labels.append(tracer.y[0])
    
    labels_points.x, labels_points.y = x_labels, y_labels
    labels_points.names = np.append(labels_points.names, label)

In [17]:
def get_new_label(labels):
    alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    alpha2 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    count = 0
    i = 0
    while i < len(alpha):
        
        elem = alpha[i]
        if not np.isin(elem,labels):
            return elem
        
        if i == (len(alpha) - 1):
            alpha = [alpha2[count] + elem for elem in alpha2]
            i = 0
            count = count + 1
            
        i = i + 1

In [18]:
def add_label_button_clicked(a):
    label = label_input.value
    if label == '':
        label = get_new_label(labels_points.names)
    add_label(tracer.x[0], tracer.y[0], label)
    label_input.value = ''

In [19]:
def update_isopressure(change):

    p = p_slider.value
    T = T_slider.value
    
    for mark in fig.marks:
        
        if mark.labels[0] == str(T):

            x_values = mark.x
            y_values = mark.y
            break
        
    roots = get_roots(p, T)
    
    if len(roots) == 3:
        
        isopressure.fill = 'between'
        shade_areas(change)
        
        #find where the isopressure curve intersects the isotherm
        i_min = find_nearest_index(x_values, roots[0]) 
        i_max = find_nearest_index(x_values, roots[2])
        
        x_real = x_values.tolist()[i_min:i_max]
        
        #we have two lines: oen for the isopressure and the other for the isotherm
        x = [x_real, x_real]
        y = [[p for elem in x_real],
             mark.y[i_min:i_max]]
        
        v_range_1 = (roots[0], roots[1])
        v_range_2 = (roots[1], roots[2])

        left_integral = defined_integral(p, v_range_1, T)
        right_integral = defined_integral(p, v_range_2, T)
        
        integral_value_left_text.value = '%.2f' % left_integral
        integral_value_right_text.value = '%.2f' % right_integral
        
        integral_value_text.value = '%.2f' % (left_integral + right_integral)
        
    else:
        isopressure.fill = 'none'
        x = [x_values, x_values]
        y = [[p for elem in x_values], mark.y]
        
        integral_value_left_text.value = '-'
        integral_value_right_text.value = '-'
        integral_value_text.value = '-'

    isopressure.x, isopressure.y = x, y

In [20]:
def get_roots(p, T):
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root > 0:
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    return roots_in_range

In [21]:
def p_undefined_integral(p_0, v_0, T):
    
    return 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0

In [22]:
def defined_integral(p_0, v_range, T):
    v_0, v_1 = v_range[0], v_range[1]
    return p_undefined_integral(p_0, v_1, T) - p_undefined_integral(p_0, v_0, T)

In [23]:
def find_real_fixed_T(p_range, T):
    
    eps = 1e-3 #1.0/p_range.size
    
    for p in p_range:
        
        roots = get_roots(p, T)
        if len(roots) == 3:
            v_range = (roots[0], roots[2])
            area = defined_integral(p, v_range, T)
            if abs(area) < eps:
                return p, v_range

    return None

In [24]:
def shade_areas(change):
    
    if shade_areas_checkbox.value:
        
        isopressure.fill_opacities = [0.35]
        
    else:
        
        isopressure.fill_opacities = [0.0]

In [25]:
def find_real_isotherm(a):
    
    maxwell_p.continuous_update = False
    
    p_min = maxwell_p.min
    p_max = maxwell_p.max
    step = maxwell_p.step
    
    p_range = np.arange(p_max, p_min, -step)
       
    for p in p_range:
        maxwell_p.value = p
        if integral_value_text.value == '0.0':
            break
            

In [26]:
def prepare_export(a):
    global fig_str
    fig_str = repr(fig)
    %store fig_str
    export_button.value = "<form action=" + "../../../apps/modules/export_module.ipynb" + " target='_blank'><button type=''submit''>Open in export module</button></form>"


In [27]:
def experimental_isotherms(t_range, v_range):
    
    theo_data = theoretical_isotherms(t_range, v_range)
    expe_data = {}
    
    v_limits = {}
    p_limits = {}
    
    v = theo_data.get('v')
    
    for t in theo_data.keys():
        
        if t != 'v':
            if float(t) < 1.0:
                try:
                    p_theo = theo_data.get(t)
                    p_expe = []
                    #p_max = scipy.signal.argrelmax(p)
                    #print(p_max)
                    #p_min = scipy.signal.argrelmin(p)
                    #print(p_min)
                    p_range = np.linspace(0.001, 1.0, num=5000)
                    p, v_range = find_real_fixed_T(p_range, t)

                    for i in range(len(v)):
                        if v[i] > v_range[0] and v[i] < v_range[1]:
                            p_expe.append(p)
                            
                        else:
                            p_expe.append(p_theo[i])
       
                    expe_data[t] = p_expe
                    v_limits[t] = [v_range[0], v_range[1]]
                    p_limits[t] = [p, p]
                    
                except: # Exception as e:
                    
                    expe_data[t] = []
                    v_limits[t] = []
                    p_limits[t] = []
            else:
                expe_data[t] = theo_data.get(t)
                
                if float(t) == 1.0:
                    v_limits[t] = [1.0]
                    p_limits[t] = [1.0]
                
                else:
                    v_limits[t] = []
                    p_limits[t] = []
                
    return expe_data, theo_data, v_limits, p_limits

In [28]:
p_range = np.linspace(0.2, 1.0, num=1000)
find_real_fixed_T(p_range, 1.25)

def get_real_isotherm_limits(v, theo_isot, expe_isot):
    #takes two arrays and gets the two points of the limit of the coexistence zone
    
    delta = theo_isot - expe_isot
    i_initial = 0
    i_final = 0
    
    for i in range(len(delta)):
        if delta[i] != 0.0:
            i_initial = i
            break
    
    for i in range(len(delta), 0, -1):
        if delta[i] != 0.0:
            i_final = i
            break
    
    v_array = [v[i_initial], v[i_final]]
    p_array = [theo_isot[i_initial], theo_isot[i_final]]
    
    return v_array, p_array

In [143]:
def generate_isotherm_marks(t_range, v_range, scale_x, scale_y):
    
    data = experimental_isotherms(t_range, v_range)
    
    expe_data = data[0]
    theo_data = data[1]
    v_limits = data[2]
    p_limits = data[3]
       
    marks = []
    
    size = len(theo_data.keys()) - 1
    step = int(np.around([255/size], 0)[0])

    c1 = 255
    c2 = 0
    c3 = 0
    
    v_theo = theo_data.get('v')
    
    for t in theo_data.keys():
        if t != 'v':

            color = '#%02x%02x%02x' % (c1, c2, c3)

            marks.append(bqm.Lines(
                #name = str(t),
                x = v_theo, 
                y = theo_data.get(t), 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [opacity_slider.value],
                visible = theoretical_toggle.value, #True, #t == '1.00',
                colors = [color],
                labels = [str(t)],
                ))
            
            if len(expe_data.get(t)) != 0:
            
                p = expe_data.get(t)
                v_real = v_theo
            
            else:
                p = [1.0]
                v_real = []
            
            marks.append(bqm.Lines(
            #    name = str(t),
                x = v_real, 
                y = p, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [opacity_slider.value],
                visible = experimental_toggle.value, #False, #t == '1.00',
                colors = [color],
                labels = [str(t)],
                line_style = 'dashed'
                ))
            
            #v_limits, p_limits = get_real_isotherm_limits(v, theo_isot, expe_isot)
            
            marks.append(bqm.Scatter(
                x = v_limits.get(t),
                y = p_limits.get(t),
                scales = {'x': scale_x, 'y': scale_y}, 
                default_opacities = [opacity_slider.value],
                visible = show_limits_checkbox.value, #False,
                colors = [color],
                names = [],
                labels = [str(t)]
            )) 
            
            
            c2 = c2 + step
    
    labels_points.x = []
    labels_points.y = []
    labels_points.names = []
    
    tracer = fig.marks[-3]
    labels_points = fig.marks[-2]
    isopressure = fig.marks[-1]
    
    marks.append(tracer)
    marks.append(labels_points)
    marks.append(isopressure)
    
    return marks

In [31]:
generate_isotherms(None)

NameError: name 'generate_isotherms' is not defined

In [32]:
T_slider

NameError: name 'T_slider' is not defined

In [33]:
widgets.HBox([fig, T_slider])

NameError: name 'fig' is not defined

In [34]:
def generate_isotherms(a):
    
    v_range = np.linspace(start=0.5, stop=5.0, num=500)
    t_range = get_t_range(t_min_input, t_max_input, num_input)
    
    fig.marks = generate_isotherm_marks(t_range, v_range, scale_x, scale_y)
    
    T_slider.options = t_range
    T_slider.value=t_range[0]
    T_slider.value=t_range[int(t_range.size/2)]
    
    select_visible.options = [str(t) for t in T_slider.options[::-1]]
    select_visible.value = [str(t) for t in T_slider.options[::-1]]
    
    select_isotherm(None)

In [127]:
tracer.labels[0] in ('tracer')

True

In [131]:
def select_isotherm(change):
    
    for mark in fig.marks:
        if mark.labels[0] == str(T_slider.value):
            mark.opacities = [1.0]
            mark.default_opacities = [1.0]
            
        else:
            if mark.labels[0] not in ('isopressure', 'labels', 'tracer'):
                mark.opacities = [opacity_slider.value]
                mark.default_opacities = [opacity_slider.value]
                
    if maxwell_construction_checkbox.value:
        
        update_isopressure(change)
        
    if show_tracer_checkbox.value:
        update_tracer(change)

In [137]:
labels_points.labels[0] in ('labels')

True

In [138]:
def show_isotherm(change):
    
    for i in range(len(fig.marks)):
        
        mark = fig.marks[i]
        label = mark.labels[0]
        
        if label not in ('isopressure', 'labels', 'tracer'):
        
            if label not in select_visible.value:
                mark.visible = False

            else:

                if i%3 == 0 and label in select_visible.value:
                    mark.visible = theoretical_toggle.value

                elif i%3 == 1 and label in select_visible.value:
                    mark.visible = experimental_toggle.value

                elif i%3 == 2 and label in select_visible.value:
                    mark.visible = show_limits_checkbox.value
                #mark.visible = True

In [37]:
def show_all(a):
    select_visible.value = [str(t) for t in T_slider.options]

In [38]:
def show_maxwell(change):
    
    if maxwell_construction_checkbox.value:
        p_slider.disabled = False
        shade_areas_checkbox.disabled = False
        
        isopressure.visible = True
        update_isopressure(change)
        
    else:
        p_slider.disabled = True
        shade_areas_checkbox.disabled = True
        isopressure.visible = False

In [89]:
def show_limits(change):
    
    t_selected = [str(t) for t in select_visible.value]
    
    for mark in fig.marks:
        if mark.labels[0] in t_selected and type(mark) == bqm.Scatter:
            mark.visible = show_limits_checkbox.value


In [40]:
fig.marks[-1]

NameError: name 'fig' is not defined

In [41]:
for mark in fig.marks:
    print(type(mark))

NameError: name 'fig' is not defined

In [42]:
def change_isotherm_type(change):
    
    t_options = [str(t) for t in T_slider.options]
    
    for i in range(len(fig.marks)):
        
        mark = fig.marks[i]
        
        if i%3 == 0 and mark.labels[0] in select_visible.value:
            mark.visible = theoretical_toggle.value
            
        elif i%3 == 1 and mark.labels[0] in select_visible.value:
            mark.visible = experimental_toggle.value

In [144]:
"""

.. module:: p_v_2D.ipynb
    :sypnopsis: This module creates an interface to interact with the
    Van der Waals isotherms in the p(v, T) plane.\n

.. moduleauthor:: Jon Gabirondo López (jgabirondo001@ikasle.ehu.eus)

"""

#%store -d fig_str
fig_str = ''

saved = []

top_block = widgets.VBox([])

t_min_input = widgets.BoundedFloatText(
    value=0.5,
    min=0.5,
    max=3.0,
    step=0.005,
    description=r'\( T_{min}: \)',
    disabled=False
)

t_max_input = widgets.BoundedFloatText(
    value=2.0,
    min=0.5,
    max=3.0,
    step=0.005,
    description=r'\( T_{max}: \)',
    disabled=False
)

#t_min_input.observe(update_t_range, 'value')
#t_max_input.observe(update_t_range, 'value')

num_input = widgets.IntSlider(
        value=5,
        min=1,
        max=20,
        description=r'\( \# \)',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        #readout_format='.2f',
    )

generate_button = widgets.Button(
        description='Generate',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Generate isotherms in the selected T range.',
    )

generate_button.on_click(generate_isotherms)

top_block.children = (widgets.HBox([
                        t_min_input,
                        t_max_input,
                        num_input,
                        generate_button]),)

t_limits = (0.9, 1.0)
v_limits = (-0.4, 2.0) #v in 10^v_limits range
v_range = np.linspace(start=0, stop=10, num=500)
t_range = get_t_range(t_min_input, t_max_input, num_input)

#T_c_index, = np.where(t_range == 1.0)[0]
#data = isotherms(t_range, v_range)

scale_x = bqs.LinearScale(min = 0.45, max = 5.0)
scale_y = bqs.LinearScale(min = 0.0, max = 2.0)

fig = bq.Figure(title='Van der Waals isotherms',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=60, right=30),
                toolbar = True
    )

axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=[0,10,100],
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig.axes = [axis_x, axis_y]

#isotherm_lines = []

marks = []

#for mark in marks:
#    mark.scales = {'x': scale_x, 'y': scale_y}

tracer = bqm.Scatter(
    name = 'tracer',
    x = [1.0],
    y = [1.0],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = False,
    colors = ['black'],
    names = [],
    labels=['tracer']
)        

marks.append(tracer)

labels_points = bqm.Scatter(
    name = 'labels',
    x = [],
    y = [],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    names = [],
    labels=['labels']
) 

marks.append(labels_points)



isopressure = bqm.Lines(
            x = [], 
            y = [], 
            scales = {'x': scale_x, 'y': scale_y}, 
            opacities = [1.0, 0.0],
            visible = False,
            colors = ['green'],
            fill_opacities = [0.35],
            fill = 'none',
            labels = ['isopressure'],
            )

marks.append(isopressure)



#labels = bqm.Label(
#    x=[],
#    y=[],
#    scales={'x': scale_x, 'y': scale_y},
#    text=[],
#    labels=['labels'],
#    default_size=14,
#    font_weight='bolder',
#    colors=['black'],
#    update_on_move=True,
#    enable_move = True)
#        
#marks.append(labels)    
    
#fig.marks = marks #isotherm_lines
#fig.marks[T_c_index].visible = True
#fig.marks[T_c_index].display_legend = True

p_c_text = widgets.HTML(
    value="",
)

v_c_text = widgets.HTML(
    value="",
)
T_c_text = widgets.HTML(
    value="",
)

#update_critics(None)

theoretical_toggle = widgets.ToggleButton(
    value=True,
    description='Theoretical',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show theoretical isotherms',
    style = {'description_width': 'initial'},
    layout = widgets.Layout(align_self='center', width='100%')
)

experimental_toggle = widgets.ToggleButton(
    value=False,
    description='Experimental',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show experimental isotherms',
    style = {'description_width': 'initial'},
    layout = widgets.Layout(align_self='center', width='100%'),
)

theoretical_toggle.observe(change_isotherm_type, 'value')
experimental_toggle.observe(change_isotherm_type, 'value')



show_limits_checkbox = widgets.Checkbox(
    value=False,
    description='Show limits',
    disabled=False,
    layout = widgets.Layout(width='148px !important')
)

show_limits_checkbox.observe(show_limits, 'value')

isotherm_type_block = widgets.HBox([],
                                  layout = widgets.Layout(align_self='center',
                                                         margin='10px 0 0 0'))

isotherm_type_block.children = (theoretical_toggle,
                                experimental_toggle,
                                show_limits_checkbox)

T_slider = widgets.SelectionSlider(
    options=t_range,
    value=t_range[int(t_range.size/2)],
    description=r'\( \frac{T}{T_c} \)',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    layout = widgets.Layout(height = '80%', margin = '45px 0 0 0')
)

T_slider.observe(select_isotherm, 'value')

select_visible = widgets.SelectMultiple(
    options=[str(t) for t in t_range],
    value=[str(t) for t in t_range],
    rows=10,
    description='',
    disabled=False,
    layout = widgets.Layout(width = '90%')
)

select_visible.observe(show_isotherm, 'value')

#T_r = widgets.FloatSlider(
#        value=1.0,
#        min=min(t_range),
#        max=max(t_range),
#        step=0.05,
#        description=r'\( \frac{T}{T_c} \)',
#        disabled=False,
#        continuous_update=True,
#        orientation='vertical',
#        readout=True,
#        readout_format='.2f',
#        layout = widgets.Layout(height = '80%', margin = '45px 0 0 0')
#    )
#
#T_r.observe(update_isotherm, 'value')

show_all_button = widgets.Button(
    description='Show all',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show all isotherms',
)

show_all_button.on_click(show_all)

opacity_slider = widgets.FloatSlider(
        value=0.2,
        min=0.0,
        max=1.0,
        step=0.05,
        description="",
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        layout = widgets.Layout(width = '90%')
        #readout_format='.2f',
    )

opacity_slider.observe(select_isotherm, 'value')

right_block = widgets.VBox([widgets.Label(value="Select visible isotherms:"),
                            select_visible,
                            show_all_button,
                            opacity_slider],
                            layout = widgets.Layout(width='15%', margin = '70px 0 0 0'))

#
#
#save_button = widgets.Button(
#        description='Save isotherm',
#        disabled=False,
#        button_style='', # 'success', 'info', 'warning', 'danger' or ''
#        tooltip='Click me',
#    )
#
#save_button.on_click(save_button_clicked)
#
#undo_button = widgets.Button(
#        description='Undo',
#        disabled=False,
#        button_style='', # 'success', 'info', 'warning', 'danger' or ''
#        tooltip='Click me',
#    )
#
#undo_button.on_click(undo)
#
#
#
maxwell_construction_checkbox = widgets.Checkbox(
    value=False,
    description='Maxwell Construction',
    disabled=False
)

maxwell_construction_checkbox.observe(show_maxwell, 'value')

p_slider = widgets.FloatSlider(
        value=1.0,
        min=0.001,
        max=scale_y.max,
        step=0.001,
        description=r'\( p_r \)',
        disabled=True,
        continuous_update=True,
        orientation='vertical',
        readout=True,
        readout_format='.3f',
        layout = widgets.Layout(height = '80%', margin = '45px 0 0 0')
    )

p_slider.observe(update_isopressure, 'value')
#
shade_areas_checkbox = widgets.Checkbox(
    value=False,
    description='shade areas',
    disabled=True
)

shade_areas_checkbox.observe(shade_areas, 'value')
#
integral_value_text = widgets.HTML(
    value="",
)

integral_value_left_text = widgets.HTML(
    value="",
) 

integral_value_right_text = widgets.HTML(
    value="",
) 

left_block = widgets.VBox([maxwell_construction_checkbox,
                                shade_areas_checkbox,
                                widgets.HBox([widgets.Label(value="$Left:$"),
                                                                integral_value_left_text,]),
                                                  widgets.HBox([widgets.Label(value="$Right:$"),
                                                                integral_value_right_text,]),
                                                    widgets.HBox([widgets.Label(value="$Sum:$"),
                                                                integral_value_text,]),
                               ],
                         layout = widgets.Layout(width='15%', margin = '70px 0 0 0'))


show_tracer_checkbox = widgets.Checkbox(
    value=False,
    description='Show tracer',
    disabled=False
)

show_tracer_checkbox.observe(show_tracer, 'value')

v_slider = widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=5.0,
        step=0.001,
        description=r'\( v \)',
        disabled=True,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(width = '90%')#, margin = '45px 0 0 0')
    )

v_slider.observe(update_tracer, 'value')

label_input = widgets.Text(
    value='',
    placeholder="Label:",
    disabled = True,
)

add_label_button = widgets.Button(
        description='Add label',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip="Add label in tracer's position",
    )

add_label_button.on_click(add_label_button_clicked)

tracer_block = widgets.VBox([show_tracer_checkbox,
                             v_slider,
                            widgets.HBox([label_input, add_label_button])],
                           layout=widgets.Layout(align_content="center"))

#
#find_real_isotherm_button = widgets.Button(
#        description='Find real isotherm',
#        disabled=False,
#        button_style='', # 'success', 'info', 'warning', 'danger' or ''
#        tooltip='Click me',
#    )
#
#find_real_isotherm_button.on_click(find_real_isotherm)
#
#prepare_export_button = widgets.Button(
#        description='Prepare to export',
#        disabled=False,
#        button_style='',
#        tooltip='',
#    )
#
#prepare_export_button.on_click(prepare_export)
#
#export_button = widgets.HTML(
#    value = ""
#)
#
#
##fig = update_figure(None)

fig.marks = marks
generate_isotherms(None)

figure_block = widgets.VBox([widgets.HBox([p_slider, fig, T_slider]),
                            tracer_block],
                            layout = widgets.Layout(width='70%'))#, margin = '70px 0 0 0'))

widgets.VBox([
    #widgets.HBox([
        #widgets.VBox([widgets.HBox([a, widgets.HTMLMath(value=r'\( \text{J} \frac{m^3}{mol^2} \)')]),
        #              widgets.HBox([b, widgets.HTMLMath(value=r'\( 10^{-3} \frac{m^3}{mol} \)')])]),
        #widgets.VBox([widgets.HBox([t_min_input, widgets.HTMLMath(value=r'\( \quad T_c \)')]),
        #              widgets.HBox([t_max_input, widgets.HTMLMath(value=r'\( \quad T_c \)')])])],
        #layout = widgets.Layout(align_self = 'center')),

    #widgets.HBox([widgets.Label(value="$p_c$:"), p_c_text, widgets.Label(value=r'\( \text{MPa} \qquad \)'),
    #              widgets.Label(value="$v_c$:"), v_c_text, widgets.Label(value=r'\( \frac{cm^3}{mol} \qquad\)'),
    #              widgets.Label(value="$T_c$:"), T_c_text, widgets.Label(value=r'\( \text{K} \qquad\)')
    #             ], layout = widgets.Layout(align_self = 'center')),
    top_block,
    isotherm_type_block,
    widgets.HBox([left_block,
                  figure_block,
                  right_block
                 ]),
    #widgets.HBox([fig,
    #              T_r,
    #              widgets.VBox(
    #                  [show_all_checkbox, save_button, undo_button,],
    #                  layout = widgets.Layout(margin = '20% 0 0 0') )
#
    #             ], layout = widgets.Layout(width = '100%')),
    #widgets.HBox([
    #    widgets.VBox([show_tracer_checkbox,
    #                  tracer_v,
    #                 widgets.HBox([label_input, add_label_button], 
    #                              layout = widgets.Layout(align_self = 'center')
    #                             )],
    #                 layout = widgets.Layout(#align_self = 'center',
    #                     margin_left = '7px',
    #                     margin_right = '7px',
    #                     width='70%')),
    #    widgets.VBox([maxwell_construction_checkbox,
    #                  widgets.HBox([maxwell_p,
    #                                 widgets.VBox([shade_areas_checkbox,
    #                                              widgets.HBox([widgets.Label(value="$Left:$"),
    #                                                            integral_value_left_text,]),
    #                                              widgets.HBox([widgets.Label(value="$Right:$"),
    #                                                            integral_value_right_text,]),
    #                                                widgets.HBox([widgets.Label(value="$Sum:$"),
    #                                                            integral_value_text,]),
    
    #                                              find_real_isotherm_button,])
    #                    ],
    #                 layout = widgets.Layout(#align_self = 'center',
    #                                                                        margin_left = '7px')),
    #]), ]),
    #widgets.HBox([prepare_export_button, export_button])
    ])
    #widgets.HBox([png_name, export_png_button,], layout = widgets.Layout(align_self = 'center',
                                                     #                   margin_bottom = '15px')),
    #widgets.HBox([png_created_text], )]), ])

UnboundLocalError: local variable 'labels_points' referenced before assignment

In [142]:
labels_points

Scatter(colors=['black'], interactions={'hover': 'tooltip'}, labels=['labels'], names=array([], dtype=float64), scales={'x': LinearScale(max=5.0, min=0.45), 'y': LinearScale(max=2.0, min=0.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9})

In [44]:
for mark in fig.marks:
    print(type(mark))

<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Scatter'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Scatter'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Scatter'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Scatter'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Lines'>
<class 'bqplot.marks.Scatter'>
<class 'bqplot.marks.Lines'>


In [97]:
isopressure.visible

False

In [46]:
theoretical_toggle

ToggleButton(value=True, description='Theoretical', layout=Layout(align_self='center', width='100%'), style=De…

In [47]:
show_limits_checkbox

Checkbox(value=False, description='Show limits', layout=Layout(width='148px !important'))

In [48]:
T_slider.value

1.25

In [49]:
fig.marks[0].labels[0]

'0.5'

In [50]:
T_slider

SelectionSlider(description='\\( \\frac{T}{T_c} \\)', index=2, layout=Layout(height='80%', margin='45px 0 0 0'…

In [51]:
top_block

In [52]:
fig

Figure(animation_duration=500, axes=[Axis(grid_color='#8e8e8e', grid_lines='none', label='v [log]', label_offs…

In [134]:
labels_points

Scatter(colors=['black'], interactions={'hover': 'tooltip'}, labels=['labels'], names=array(['fdsa', 'A'], dtype='<U32'), scales={'x': LinearScale(max=5.0, min=0.45), 'y': LinearScale(max=2.0, min=0.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, visible=False, x=array([1.3747495, 1.3747495]), y=array([1.61341454, 1.61341454]))

In [54]:
fig.marks[21]

IndexError: list index out of range

In [ ]:
get_t_range(t_min_input, t_max_input, num_input)

In [ ]:
data.get(t_min_input.value)

In [ ]:
t_min_input.value

In [ ]:
fig

In [ ]:
fig.marks[0].name

In [ ]:
scale_y